In [ ]:
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from raster_file import RasterFile

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap
from common import *
from reduce_grid import get_mask

plt.rcParams.update(
    {
        'xtick.labelsize' : 10,
        'ytick.labelsize' : 10,
    }
)

## Snow Depth comparison 

In [ ]:
FILE_PREFIX = 'Depth_difference_'
HOME = Path.home() / 'scratch/iSnobal/ASO-data/'
DATA = HOME / 'Depth-Difference'

# Values above 25m are considered outlier
OUTLIER = 25

In [ ]:
def violin_plot(ax, data, color):
    vp = ax.violinplot(
        data,
        showmeans=False, showextrema=False, showmedians=True,
    )
    ax.set_xticks([])
    ax.axhline(0, c='black', ls=':')
    
    for pc in vp['bodies']:
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
    
    vp['cmedians'].set_color('black')


In [ ]:
def area_plot(date, differences, save_figure=False):
    data = differences.band_values()
    
    data[data > OUTLIER] = np.NaN
    
    fig, (ax1, ax2) = plt.subplots(
        1, 2, figsize=(11.5, 9),
        gridspec_kw={'width_ratios': [82, 18], 'wspace': 0.1},
        dpi=500,
    )
    fig.set_facecolor('lightgrey')
    ax1.set_facecolor('slategrey')

    plt_data = ax1.imshow(
        data,
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP
    )

    ax1.tick_params(axis='both', direction='inout', length=7, labelsize=9, pad=0)
    ax1.tick_params(axis='y', labelrotation=45)
    ax1.ticklabel_format(axis='both', style='plain')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
    
    cax = make_axes_locatable(ax1).append_axes('right', size='5%', pad=0.15)
    fig.colorbar(plt_data, cax=cax, shrink=0.9, extend='both', label=r'$\Delta$ Depth (m)')

    mask = get_mask(differences.file.RasterYSize, differences.file.RasterXSize)
    violin_data = data.filled(np.NaN)[mask]

    violin_data = violin_data[~np.isnan(violin_data)]
    violin_plot(ax2, violin_data, 'orange')
    ax2.set_ylabel(None)
    ax2.set_ylim([-3, 4])
    ax2.tick_params(axis='y', labelsize=9)
    
    ax2.text(
        0.05, 0.01, 
        f'Median: {np.median(violin_data):.2f}',
        transform=ax2.transAxes, 
        verticalalignment='bottom', 
        bbox=dict(
            facecolor='whitesmoke', 
            alpha=0.4, 
            pad=.3, 
            boxstyle='round'
        ),
    )
    
    if save_figure:
        plt.savefig(f"{HOME}/figures/ASO_depth_difference_{date}.png")
    
    del data
    del violin_data
    plt.clf()


In [ ]:
dates = [
    '20180331',
    '20180524',
    '20190407',
    '20190610',
]

for date in dates:
    snow_nc = f'{FILE_PREFIX}{date}_1m.tif'
    differences = RasterFile(DATA.joinpath(snow_nc).as_posix())    
    area_plot(date, differences, True)